In [1]:
import itertools
import numpy as np
import pandas as pd
# for data scaling and splitting
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
# for neural net
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# for evaluation
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data = pd.read_csv("data/combined_expression.csv")
data.head()
data.shape

(642, 16383)

In [3]:
selected_genes = pd.read_csv('cleaned/boruta-99-25-0.01.csv')
selected_genes = selected_genes.values.tolist()
selected_genes = list(itertools.chain(*selected_genes))

In [4]:
# retrieving proper columns
X = data.loc[:, selected_genes]
y = data['classification'].values

# scaling the data
scalar = MinMaxScaler()
x_scaled = scalar.fit_transform(X)

# splitting data (20% test, 80% train)
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=0)

# 3 Hidden Layers

In [37]:
def create_model(optimizer='rmsprop',init='glorot_uniform', dropout=0.3):
    model = Sequential()
    # adding layers and adding droplayers to avoid overfitting
    hidden_layers = len(selected_genes)
    model.add(Dense(hidden_layers, activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.5), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.25), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [38]:
# parameters selected from previous gridsearch
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=64, optimizer='Adagrad',init='uniform')
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Baseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 3ms/sample - loss: 0.6833 - accuracy: 0.5439
Epoch 2/50
342/342 [==============================] - 0s 141us/sample - loss: 0.6453 - accuracy: 0.6257
Epoch 3/50
342/342 [==============================] - 0s 143us/sample - loss: 0.6522 - accuracy: 0.6404
Epoch 4/50
342/342 [==============================] - 0s 141us/sample - loss: 0.6304 - accuracy: 0.6725
Epoch 5/50
342/342 [==============================] - 0s 142us/sample - loss: 0.6150 - accuracy: 0.6608
Epoch 6/50
342/342 [==============================] - 0s 140us/sample - loss: 0.6278 - accuracy: 0.6579
Epoch 7/50
342/342 [==============================] - 0s 143us/sample - loss: 0.6025 - accuracy: 0.6667
Epoch 8/50
342/342 [==============================] - 0s 146us/sample - loss: 0.5996 - accuracy: 0.7047
Epoch 9/50
342/342 [==============================] - 0s 142us/sample - loss: 0.6165 - accuracy: 0.6608
Epoch 10/50
342/342 [========================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 3ms/sample - loss: 0.7242 - accuracy: 0.5702
Epoch 2/50
342/342 [==============================] - 0s 144us/sample - loss: 0.6469 - accuracy: 0.6404
Epoch 3/50
342/342 [==============================] - 0s 144us/sample - loss: 0.6211 - accuracy: 0.6579
Epoch 4/50
342/342 [==============================] - 0s 146us/sample - loss: 0.6414 - accuracy: 0.6404
Epoch 5/50
342/342 [==============================] - 0s 146us/sample - loss: 0.6314 - accuracy: 0.6579
Epoch 6/50
342/342 [==============================] - 0s 141us/sample - loss: 0.6203 - accuracy: 0.6608
Epoch 7/50
342/342 [==============================] - 0s 145us/sample - loss: 0.6383 - accuracy: 0.6433
Epoch 8/50
342/342 [==============================] - 0s 142us/sample - loss: 0.6118 - accuracy: 0.6784
Epoch 9/50
342/342 [==============================] - 0s 144us/sample - loss: 0.6083 - accuracy: 0.6784
Epoch 10/50
342/342 [========================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 3ms/sample - loss: 0.6444 - accuracy: 0.6404
Epoch 2/50
342/342 [==============================] - 0s 145us/sample - loss: 0.5972 - accuracy: 0.6784
Epoch 3/50
342/342 [==============================] - 0s 145us/sample - loss: 0.6102 - accuracy: 0.6930
Epoch 4/50
342/342 [==============================] - 0s 142us/sample - loss: 0.5989 - accuracy: 0.6871
Epoch 5/50
342/342 [==============================] - 0s 142us/sample - loss: 0.5932 - accuracy: 0.6959
Epoch 6/50
342/342 [==============================] - 0s 147us/sample - loss: 0.5905 - accuracy: 0.6754
Epoch 7/50
342/342 [==============================] - 0s 147us/sample - loss: 0.6140 - accuracy: 0.6754
Epoch 8/50
342/342 [==============================] - 0s 144us/sample - loss: 0.5860 - accuracy: 0.7281
Epoch 9/50
342/342 [==============================] - 0s 148us/sample - loss: 0.5812 - accuracy: 0.6901
Epoch 10/50
342/342 [========================

Baseline Accuracy: 73.68% (1.72%)


In [39]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Train on 513 samples
Epoch 1/50
513/513 [==============================] - 1s 2ms/sample - loss: 0.6679 - accuracy: 0.6257
Epoch 2/50
513/513 [==============================] - 0s 138us/sample - loss: 0.6250 - accuracy: 0.6647
Epoch 3/50
513/513 [==============================] - 0s 140us/sample - loss: 0.6286 - accuracy: 0.6569
Epoch 4/50
513/513 [==============================] - 0s 145us/sample - loss: 0.6167 - accuracy: 0.6569
Epoch 5/50
513/513 [==============================] - 0s 144us/sample - loss: 0.6217 - accuracy: 0.6706
Epoch 6/50
513/513 [==============================] - 0s 143us/sample - loss: 0.6036 - accuracy: 0.7057
Epoch 7/50
513/513 [==============================] - 0s 146us/sample - loss: 0.6134 - accuracy: 0.6550
Epoch 8/50
513/513 [==============================] - 0s 144us/sample - loss: 0.6067 - accuracy: 0.6803
Epoch 9/50
513/513 [==============================] - 0s 146us/sample - loss: 0.6062 - accuracy: 0.6686
Epoch 10/50
513/513 [========================

In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.66      0.92      0.77        75
           2       0.75      0.33      0.46        54

    accuracy                           0.67       129
   macro avg       0.70      0.63      0.61       129
weighted avg       0.70      0.67      0.64       129



In [32]:
print(confusion_matrix(y_test, y_pred))

[[67  8]
 [32 22]]


In [14]:
model.model.save('models/hidden_3.h5')

# 2 Hidden Layers

In [15]:
def create_model(optimizer='rmsprop',init='glorot_uniform', dropout=0.3):
    model = Sequential()
    # adding layers and adding droplayers to avoid overfitting
    hidden_layers = len(selected_genes)
    model.add(Dense(hidden_layers, activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense((hidden_layers*0.5), activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [16]:
# parameters selected from previous gridsearch
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=64, optimizer='Adagrad',init='uniform')
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Baseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 3ms/sample - loss: 0.6388 - accuracy: 0.6637
Epoch 2/50
342/342 [==============================] - 0s 123us/sample - loss: 0.6100 - accuracy: 0.6842
Epoch 3/50
342/342 [==============================] - 0s 123us/sample - loss: 0.5860 - accuracy: 0.7018
Epoch 4/50
342/342 [==============================] - 0s 125us/sample - loss: 0.5731 - accuracy: 0.6988
Epoch 5/50
342/342 [==============================] - 0s 128us/sample - loss: 0.5647 - accuracy: 0.7281
Epoch 6/50
342/342 [==============================] - 0s 127us/sample - loss: 0.5718 - accuracy: 0.6988
Epoch 7/50
342/342 [==============================] - 0s 126us/sample - loss: 0.5551 - accuracy: 0.7310
Epoch 8/50
342/342 [==============================] - 0s 129us/sample - loss: 0.5326 - accuracy: 0.7310
Epoch 9/50
342/342 [==============================] - 0s 127us/sample - loss: 0.5445 - accuracy: 0.7485
Epoch 10/50
342/342 [========================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 3ms/sample - loss: 0.6668 - accuracy: 0.6257
Epoch 2/50
342/342 [==============================] - 0s 122us/sample - loss: 0.6618 - accuracy: 0.5936
Epoch 3/50
342/342 [==============================] - 0s 122us/sample - loss: 0.6375 - accuracy: 0.6491
Epoch 4/50
342/342 [==============================] - 0s 127us/sample - loss: 0.6278 - accuracy: 0.6550
Epoch 5/50
342/342 [==============================] - 0s 130us/sample - loss: 0.6253 - accuracy: 0.6345
Epoch 6/50
342/342 [==============================] - 0s 126us/sample - loss: 0.6254 - accuracy: 0.6287
Epoch 7/50
342/342 [==============================] - 0s 127us/sample - loss: 0.6178 - accuracy: 0.6345
Epoch 8/50
342/342 [==============================] - 0s 126us/sample - loss: 0.6076 - accuracy: 0.6725
Epoch 9/50
342/342 [==============================] - 0s 124us/sample - loss: 0.6181 - accuracy: 0.6696
Epoch 10/50
342/342 [========================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 3ms/sample - loss: 0.6620 - accuracy: 0.6257
Epoch 2/50
342/342 [==============================] - 0s 126us/sample - loss: 0.6110 - accuracy: 0.6667
Epoch 3/50
342/342 [==============================] - 0s 129us/sample - loss: 0.6040 - accuracy: 0.6667
Epoch 4/50
342/342 [==============================] - 0s 131us/sample - loss: 0.5976 - accuracy: 0.6813
Epoch 5/50
342/342 [==============================] - 0s 131us/sample - loss: 0.5781 - accuracy: 0.6784
Epoch 6/50
342/342 [==============================] - 0s 132us/sample - loss: 0.5615 - accuracy: 0.7018
Epoch 7/50
342/342 [==============================] - 0s 137us/sample - loss: 0.5734 - accuracy: 0.6988
Epoch 8/50
342/342 [==============================] - 0s 136us/sample - loss: 0.5638 - accuracy: 0.7047
Epoch 9/50
342/342 [==============================] - 0s 139us/sample - loss: 0.5589 - accuracy: 0.7222
Epoch 10/50
342/342 [========================

Baseline Accuracy: 74.27% (7.21%)


In [17]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Train on 513 samples
Epoch 1/50
513/513 [==============================] - 1s 2ms/sample - loss: 0.6462 - accuracy: 0.6511
Epoch 2/50
513/513 [==============================] - 0s 131us/sample - loss: 0.6009 - accuracy: 0.6686
Epoch 3/50
513/513 [==============================] - 0s 132us/sample - loss: 0.6013 - accuracy: 0.6725
Epoch 4/50
513/513 [==============================] - 0s 133us/sample - loss: 0.5945 - accuracy: 0.6745
Epoch 5/50
513/513 [==============================] - 0s 134us/sample - loss: 0.5797 - accuracy: 0.6803
Epoch 6/50
513/513 [==============================] - 0s 138us/sample - loss: 0.5849 - accuracy: 0.6823
Epoch 7/50
513/513 [==============================] - 0s 137us/sample - loss: 0.5746 - accuracy: 0.7076
Epoch 8/50
513/513 [==============================] - 0s 138us/sample - loss: 0.5808 - accuracy: 0.7018
Epoch 9/50
513/513 [==============================] - 0s 138us/sample - loss: 0.5791 - accuracy: 0.7173
Epoch 10/50
513/513 [========================

In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.67      0.91      0.77        75
           2       0.75      0.39      0.51        54

    accuracy                           0.69       129
   macro avg       0.71      0.65      0.64       129
weighted avg       0.71      0.69      0.66       129



In [19]:
print(confusion_matrix(y_test, y_pred))

[[68  7]
 [33 21]]


In [20]:
model.model.save('models/hidden_2.h5')

# 1 Hidden Layer

In [21]:
def create_model(optimizer='rmsprop',init='glorot_uniform', dropout=0.3):
    model = Sequential()
    # adding layers and adding droplayers to avoid overfitting
    hidden_layers = len(selected_genes)
    model.add(Dense(hidden_layers, activation='relu'))
    model.add(Dropout(dropout))
    
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [23]:
# parameters selected from previous gridsearch
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=64, optimizer='Adagrad',init='uniform')
kfold = KFold(n_splits=3, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Baseline Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 2ms/sample - loss: 0.7090 - accuracy: 0.5614
Epoch 2/50
342/342 [==============================] - 0s 98us/sample - loss: 0.6379 - accuracy: 0.6579
Epoch 3/50
342/342 [==============================] - 0s 98us/sample - loss: 0.6177 - accuracy: 0.6667
Epoch 4/50
342/342 [==============================] - 0s 97us/sample - loss: 0.6061 - accuracy: 0.6608
Epoch 5/50
342/342 [==============================] - 0s 102us/sample - loss: 0.5950 - accuracy: 0.6725
Epoch 6/50
342/342 [==============================] - 0s 99us/sample - loss: 0.6000 - accuracy: 0.6754
Epoch 7/50
342/342 [==============================] - 0s 100us/sample - loss: 0.5826 - accuracy: 0.7105
Epoch 8/50
342/342 [==============================] - 0s 100us/sample - loss: 0.5896 - accuracy: 0.6813
Epoch 9/50
342/342 [==============================] - 0s 97us/sample - loss: 0.5772 - accuracy: 0.6813
Epoch 10/50
342/342 [=============================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 2ms/sample - loss: 0.6836 - accuracy: 0.6023
Epoch 2/50
342/342 [==============================] - 0s 97us/sample - loss: 0.6214 - accuracy: 0.6784
Epoch 3/50
342/342 [==============================] - 0s 101us/sample - loss: 0.6072 - accuracy: 0.6871
Epoch 4/50
342/342 [==============================] - 0s 103us/sample - loss: 0.6039 - accuracy: 0.6842
Epoch 5/50
342/342 [==============================] - 0s 104us/sample - loss: 0.5949 - accuracy: 0.6813
Epoch 6/50
342/342 [==============================] - 0s 103us/sample - loss: 0.5951 - accuracy: 0.6842
Epoch 7/50
342/342 [==============================] - 0s 99us/sample - loss: 0.5769 - accuracy: 0.6842
Epoch 8/50
342/342 [==============================] - 0s 102us/sample - loss: 0.5672 - accuracy: 0.7251
Epoch 9/50
342/342 [==============================] - 0s 103us/sample - loss: 0.5640 - accuracy: 0.6988
Epoch 10/50
342/342 [==========================

Train on 342 samples
Epoch 1/50
342/342 [==============================] - 1s 3ms/sample - loss: 0.6816 - accuracy: 0.5702
Epoch 2/50
342/342 [==============================] - 0s 96us/sample - loss: 0.6308 - accuracy: 0.6520
Epoch 3/50
342/342 [==============================] - 0s 102us/sample - loss: 0.6148 - accuracy: 0.6462
Epoch 4/50
342/342 [==============================] - 0s 100us/sample - loss: 0.6082 - accuracy: 0.6901
Epoch 5/50
342/342 [==============================] - 0s 105us/sample - loss: 0.6147 - accuracy: 0.6637
Epoch 6/50
342/342 [==============================] - 0s 107us/sample - loss: 0.5963 - accuracy: 0.6754
Epoch 7/50
342/342 [==============================] - 0s 104us/sample - loss: 0.5871 - accuracy: 0.6813
Epoch 8/50
342/342 [==============================] - 0s 103us/sample - loss: 0.5919 - accuracy: 0.6930
Epoch 9/50
342/342 [==============================] - 0s 104us/sample - loss: 0.5736 - accuracy: 0.7018
Epoch 10/50
342/342 [=========================

Baseline Accuracy: 71.54% (1.81%)


In [24]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Train on 513 samples
Epoch 1/50
513/513 [==============================] - 1s 1ms/sample - loss: 0.6378 - accuracy: 0.6550
Epoch 2/50
513/513 [==============================] - 0s 108us/sample - loss: 0.6064 - accuracy: 0.6764
Epoch 3/50
513/513 [==============================] - 0s 109us/sample - loss: 0.6049 - accuracy: 0.6803
Epoch 4/50
513/513 [==============================] - 0s 111us/sample - loss: 0.5766 - accuracy: 0.6940
Epoch 5/50
513/513 [==============================] - 0s 103us/sample - loss: 0.5789 - accuracy: 0.7037
Epoch 6/50
513/513 [==============================] - 0s 104us/sample - loss: 0.5901 - accuracy: 0.6803
Epoch 7/50
513/513 [==============================] - 0s 103us/sample - loss: 0.5871 - accuracy: 0.7251
Epoch 8/50
513/513 [==============================] - 0s 105us/sample - loss: 0.5732 - accuracy: 0.6940
Epoch 9/50
513/513 [==============================] - 0s 106us/sample - loss: 0.5794 - accuracy: 0.6979
Epoch 10/50
513/513 [========================

In [25]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.67      0.93      0.78        75
           2       0.79      0.35      0.49        54

    accuracy                           0.69       129
   macro avg       0.73      0.64      0.63       129
weighted avg       0.72      0.69      0.66       129



In [26]:
print(confusion_matrix(y_test, y_pred))

[[70  5]
 [35 19]]


In [27]:
model.model.save('models/hidden_1.h5')